In [1]:
using JuMP, ConditionalJuMP, Gurobi

In [7]:
m = Model(solver=GurobiSolver())
@variable m -1 <= x <= 1
@variable m -1 <= y <= 1
@implies m (x <= 0) => (y == 0.5)
@constraint m y >= x
d = -1
@objective m Min ((x - d)^2 + (y - (d - 0.5))^2)
solve(m)
getvalue(x), getvalue(y)

Optimize a model with 6 rows, 3 columns and 12 nonzeros
Model has 2 quadratic objective terms
Variable types: 2 continuous, 1 integer (1 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+00]
  Objective range  [2e+00, 3e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 5 rows, 3 columns, 11 nonzeros
Presolved model has 2 quadratic objective terms
Variable types: 2 continuous, 1 integer (1 binary)

Root relaxation: objective -5.000000e-02, 6 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   -0.05000    0    1          -   -0.05000      -     -    0s
H    0     0                       0.0000000   -0.05000      -     -    0s

Explored 0 nodes (6 simplex iterations) in 0.00 seconds
Thread count was 12 (of 12 avail

(0.0, 0.0)

In [225]:
m = Model(solver=GurobiSolver())
@variable m x >= 0.5
@variable m y
@constraint m y >= x
# JuMP.fix(x, 0.5)
@objective m Min x^2 + y^2 + 0.1 * x * y
# c = @constraint m x == -0.5
solve(m)
display(m)
getvalue.((y, x))

Minimization problem with:
 * 1 linear constraint
 * 2 variables
Solver is Gurobi

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model has 3 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-01, 2e+00]
  Bounds range     [5e-01, 5e-01]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 3 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 1
 AA' NZ     : 1.000e+00
 Factor NZ  : 3.000e+00
 Factor Ops : 5.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.49759884e+05 -2.49262247e+05  1.02e+03 2.31e-07  1.00e+06     0s
   1   1.66279362e+05 -1.65750543e+05  1.03e-03 2.27e-13  1.11e+05     0s
   2   2.44246029e+04 -2.42216146e+04  1.02e-09 0.00e+00  1.62e+04     0s
   3   3.60264336e+03 -3.52450433e+03  3.22e-15 7.11e-15  2.38e+03     0s

(0.5000000000116488, 0.5000000000029408)

In [226]:
module X

using JuMP

# See Bemporad, "A Survey on Explicit Model Predictive Control", 
# section 2.2 (page 353). 
# struct CanonicalModel{T, M <: AbstractMatrix{T}}
#     Y::M
#     H::M
#     F::M
#     G::M
#     W::Vector{T}
#     S::M
# end

struct ExplicitSolution{T, M <: AbstractMatrix{T}}
    constant::Vector{T}
    linear::M
    x::Vector{T}
end

(s::ExplicitSolution)(x) = s.constant .+ s.linear * x
(s::ExplicitSolution)() = s.constant .+ s.linear * s.x
jacobian(s::ExplicitSolution) = s.linear

getcol(x::Variable) = x.col

function explicit_solution(m::Model, params::AbstractArray{Variable}, eps=1e-3)
    @assert isempty(m.obj.aff)
    
    nvars = length(m.colCat)
    A_active = SparseVector{Float64, Int64}[]
    b_active = Float64[]
    
    for i in 1:nvars
        var = Variable(m, i)
        if var in params
            continue
        end
        λ = getdual(var)
        if abs(λ) > eps
            if λ > 0
                ai = sparsevec([i], [-1.0], nvars)
                bi = -m.colLower[i]
            else
                ai = sparsevec([i], [1.0], nvars)
                bi = m.colUpper[i]
            end 
            push!(A_active, ai)
            push!(b_active, bi)
        end
    end
    
    nconstr = length(m.linconstr)
    for (constraint, λ) in zip(m.linconstr, m.linconstrDuals)
        if abs(λ) > eps
            if λ > 0
                ai = -sparsevec([var.col for var in constraint.terms.vars], constraint.terms.coeffs, nvars)
                bi = -constraint.lb
            else
                ai = sparsevec([var.col for var in constraint.terms.vars], constraint.terms.coeffs, nvars)
                bi = constraint.ub
            end
            push!(A_active, ai)
            push!(b_active, bi)
        end
    end
    
    param_cols = Set([v.col for v in params])
    isparam = collect(1:nvars) .∈ param_cols
    Ã = vcat(transpose.(A_active)...)
    W̃ = vcat(b_active...)
    G̃ = Ã[:, .!isparam]
    S̃ = .-Ã[:, isparam]
    
    Q = sparse(getcol.(m.obj.qvars1), getcol.(m.obj.qvars2), m.obj.qcoeffs, nvars, nvars)
    Q = 0.5 .* (Q .+ Q')
    H = Q[.!isparam, .!isparam]
    Hi = full(H)^-1
    F = Q[isparam, .!isparam]
    Y = Q[isparam, isparam]
    
    v_test = rand(nvars)
    x_test = v_test[isparam]
    z_test = v_test[.!isparam]
    @assert 0.5 * v_test' * Q * v_test ≈ (0.5 * z_test' * H * z_test + x_test' * F * z_test + 0.5 * x_test' * Y * x_test)

    x = getvalue(params)    
    λ_active = -(G̃ * Hi * G̃') * (W̃ + (S̃ + G̃ * Hi * F) * x)
    T = Hi * G̃' * (G̃ * Hi * G̃')^-1
    ExplicitSolution(T * W̃, T * (S̃ + G̃ * Hi * F) - Hi * F, x)
end

end

X

In [227]:
solution = X.explicit_solution(m, [x])

X.ExplicitSolution{Float64,Array{Float64,2}}([0.0], [1.0], [0.5])

In [228]:
solution()

1-element Array{Float64,1}:
 0.5

In [229]:
getvalue.((y, x))

(0.5000000000116488, 0.5000000000029408)

In [169]:
solution = X.canonicalize

LoadError: [91minvalid redefinition of constant ExplicitSolution[39m

canonicalize (generic function with 2 methods)

In [163]:
A

3×3 Array{Float64,2}:
 0.385527  0.0       0.0     
 0.493046  0.750508  0.0     
 0.229975  0.902256  0.605159

In [166]:
@which A^-1

^(A::AbstractArray{T,2}, p::Integer) where T in Base.LinAlg at linalg/dense.jl:332

In [165]:
inv(A)

3×3 Array{Float64,2}:
  2.59385   0.0      0.0    
 -1.70403   1.33243  0.0    
  1.55488  -1.98657  1.65246

In [162]:
canonicalize(m, [x])